In [11]:
# 모듈 임포트
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.svm import OneClassSVM
from sklearn.metrics import classification_report
import joblib
import time
import matplotlib.pyplot as plt
import glob
import os

In [12]:
# 컬럼명 정의
column_names = ['timestamp', 'v_raw', 'c_raw', 'voltage', 'current', 'label']

# 폴더 경로
# normal_dir = '../realtime/normal/'
# normal_files = glob.glob(os.path.join(normal_dir, '*.csv'))
# normal_dfs = [pd.read_csv(file, names=column_names, header=None) for file in normal_files]
# normal_data = pd.concat(normal_dfs, ignore_index=True)


# # arc 폴더 CSV 병합
# arc_dir = '../realtime/arc/'
# arc_files = glob.glob(os.path.join(arc_dir, '*.csv'))
# arc_dfs = [pd.read_csv(file, names=column_names, header=None) for file in arc_files]
# arc_data = pd.concat(arc_dfs, ignore_index=True)

# 전체 병합
# train_df = pd.concat([normal_data, arc_data], ignore_index=True)

# train_df = normal_data

# 셔플
normal_dir = '../cnn1D/log/0529/학습데이터/노트북 전용/'
normal_files = glob.glob(os.path.join(normal_dir, '*.csv'))
print(normal_files)
normal_dfs = [pd.read_csv(file, names=column_names, header=None) for file in normal_files]
train_df = pd.concat(normal_dfs, ignore_index=True)
# train_df = pd.read_csv('../cnn1d/log/0529/학습데이터/노트북 전용/정상 데이터(40초).csv', names=column_names, header=None)
train_df = train_df.sample(frac=1).reset_index(drop=True)

['../cnn1D/log/0529/학습데이터/노트북 전용/무부하 정상 데이터.csv']


In [13]:
# 파생 피처 추가 함수
def add_features(df):
    df['voltage_diff'] = df['voltage'].diff().fillna(0).abs()
    df['current_diff'] = df['current'].diff().fillna(0).abs()
    df['voltage_ma'] = df['voltage'].rolling(5).mean().bfill()
    df['current_ma'] = df['current'].rolling(5).mean().bfill()
    df['power'] = df['voltage'] * df['current']
    df['power_diff'] = df['power'].diff().fillna(0).abs()
    return df

train_df = add_features(train_df).dropna()
# test_df = add_features(test_df).dropna()

In [14]:
features = ['voltage', 'current', 'voltage_diff', 'current_diff',
            'voltage_ma', 'current_ma', 'power', 'power_diff']
X_train = train_df[features].values
# X_test = test_df[features].values
# y_true = test_df['label'].values

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
# X_test_scaled = scaler.transform(X_test)

In [ ]:
model = OneClassSVM(nu=0.005, kernel='rbf', gamma='scale')
model.fit(X_train_scaled)

joblib.dump(model, './model/one_class_svm_model_new.joblib')
joblib.dump(scaler, './model/scaler_new.joblib')

['./model/scaler_무부하.joblib']